In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

from numpy.linalg import norm, solve
import numpy as np

In [80]:
data = np.loadtxt("benchmark_data.txt").T

min_position_rk = data[0:7]
min_velocity_rk = data[7:14]
min_accel_rk = data[14:21]
min_torque_rk = data[21:28]

max_position_rk = data[28:35]
max_velocity_rk = data[35:42]
max_accel_rk = data[42:49]
max_torque_rk = data[49:56]

min_position_mpc = data[56:63]
min_velocity_mpc = data[63:70]
min_accel_mpc = data[70:77]
min_torque_mpc = data[77:84]

max_position_mpc = data[84:91]
max_velocity_mpc = data[91:98]
max_accel_mpc = data[98:105]
max_torque_mpc = data[105:112]

position_limit = np.array([[-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973], 
                          [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973]])

velocity_limit = np.array([2.1750, 2.1750, 2.1750, 2.1750, 2.6100, 2.6100, 2.6100])

acceleration_limit = np.array([15.0, 7.5, 10.0, 12.5, 15.0, 20.0, 20.0])

torque_limit = np.array([87, 87, 87, 87, 12, 12, 12])

In [82]:
fig = make_subplots(rows=7, cols=2, column_titles=["position vs velocity", "acceleration vs torque"], row_titles=["Joint 1", "Joint 2", "Joint 3", "Joint 4", "Joint 5", "Joint 6", "Joint 7"])

for i in range(7):

        # Plot ranges
        fig.add_shape(y0=-torque_limit[i], y1=torque_limit[i],
                        x0=-acceleration_limit[i], x1=acceleration_limit[i],
                        row=i+1, col=2, line_width=2, fillcolor="orange", opacity=0.2)

        fig.add_shape(y0=-0.8*torque_limit[i], y1=0.8*torque_limit[i],
                        x0=-0.7*acceleration_limit[i], x1=0.7*acceleration_limit[i],
                        row=i+1, col=2, line_width=0, fillcolor="blue", opacity=0.1)

        # Plot ranges
        fig.add_shape(y0=-velocity_limit[i], y1=velocity_limit[i],
                        x0=position_limit[0, i], x1=position_limit[1, i],
                        row=i+1, col=1, line_width=2, fillcolor="orange", opacity=0.2)

        fig.add_shape(y0=-0.8*velocity_limit[i], y1=0.8*velocity_limit[i],
                        x0=position_limit[0, i]+0.3, x1=position_limit[1, i]-0.3,
                        row=i+1, col=1, line_width=0, fillcolor="blue", opacity=0.1)

        # Plot position and velocity extremums
        fig.add_trace(
        go.Scatter(x=min_position_rk[i, :], y=min_velocity_rk[i, :], mode='markers',
                marker=dict(
                color='red',
                opacity=0.5,
                size=5)),
        row=i+1, col=1 )

        fig.add_trace(
        go.Scatter(x=max_position_rk[i, :], y=max_velocity_rk[i, :], mode='markers',
                marker=dict(
                color='red',
                opacity=0.5,
                size=5)),
        row=i+1, col=1 )


        fig.add_trace(
        go.Scatter(x=min_position_mpc[i, :], y=min_velocity_mpc[i, :], mode='markers',
                marker=dict(
                color='green',
                opacity=0.5,
                size=5)),
        row=i+1, col=1 )

        fig.add_trace(
        go.Scatter(x=max_position_mpc[i, :], y=max_velocity_mpc[i, :], mode='markers',
                marker=dict(
                color='green',
                opacity=0.5,
                size=5)),
        row=i+1, col=1 )

        # fig.add_vline(x=position_limit[0, i],
        # row=i+1, col=1, line_dash="dash", line_width=1)

        # fig.add_vline(x=position_limit[1, i],
        # row=i+1, col=1, line_dash="dash", line_width=1)

        # fig.add_hline(y=-velocity_limit[i],
        # row=i+1, col=1, line_dash="dash", line_width=1)

        # fig.add_hline(y=velocity_limit[i],
        # row=i+1, col=1, line_dash="dash", line_width=1)



        # Plot acceelartion and torque extremums
        fig.add_trace(
        go.Scatter(x=min_accel_rk[i, :], y=min_torque_rk[i, :], mode='markers',
                marker=dict(
                color='red',
                opacity=0.5,
                size=5)),
        row=i+1, col=2 )

        fig.add_trace(
        go.Scatter(x=max_accel_rk[i, :], y=max_torque_rk[i, :], mode='markers',
                marker=dict(
                color='red',
                opacity=0.5,
                size=5)),
        row=i+1, col=2 )


        fig.add_trace(
        go.Scatter(x=min_accel_mpc[i, :], y=min_torque_mpc[i, :], mode='markers',
                marker=dict(
                color='green',
                opacity=0.5,
                size=5)),
        row=i+1, col=2 )

        fig.add_trace(
        go.Scatter(x=max_accel_mpc[i, :], y=max_torque_mpc[i, :], mode='markers',
                marker=dict(
                color='green',
                opacity=0.5,
                size=5)),
        row=i+1, col=2 )



        


    
fig.update_layout(
    height=1500,
    width=1000,
    plot_bgcolor='rgba(250,250,250, 0.7)')
fig.update(layout_showlegend=False)
fig.show()

In [45]:
px.histogram(x=min_position_rk[3, :], nbins=20)

In [56]:
import plotly.express as px

df = px.data.iris()
fig = px.scatter(df, x="petal_length", y="petal_width")
fig.add_vline(x=2.5, line_width=3, line_dash="dash", line_color="green")
fig.add_shape(y0=0.9, y1=2.6, x0=1, x1=5, line_width=0, fillcolor="red", opacity=0.2)
fig.show()